# Trabalho 04 – Análise de sentimento com HuggingFace `pipeline` na base IMDB

Este notebook é para o **Trabalho 04** da disciplina de Redes Neurais Artificiais (RNA).

### Enunciado (resumo)

- Usar a classe **`pipeline` da biblioteca `transformers` do HuggingFace** para analisar o sentimento.
- Utilizar a base **IMDB** de reviews de filmes:

  - Arquivo: `aclImdb_v1.tar.gz`
  - Link: <https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz>

- A referência teórica é a parte de *Behind the pipeline* do curso da HuggingFace:

  - <https://huggingface.co/learn/nlp-course/chapter2/2?fw=pt>

A ideia é manter o estilo dos trabalhos anteriores: código simples, com alguns comentários mais
“humanos” sobre as escolhas e pequenos tropeços pelo caminho.


## 1. Base IMDB (explicação rápida)

A base **IMDB** é um conjunto de reviews de filmes em inglês, rotulados como:

- **`pos`** (review positivo)
- **`neg`** (review negativo)

O arquivo `aclImdb_v1.tar.gz` contém uma estrutura de pastas mais ou menos assim:

- `aclImdb/train/pos`
- `aclImdb/train/neg`
- `aclImdb/test/pos`
- `aclImdb/test/neg`

Cada arquivo de texto dentro dessas pastas é um review de um filme.

Para este trabalho, eu vou:

1. Fazer o **download** do arquivo `.tar.gz` a partir do link informado.
2. **Extrair** o conteúdo (train/test, pos/neg).
3. Carregar uma **amostra** de reviews (para não ficar muito pesado).
4. Usar a classe `pipeline` do HuggingFace para fazer **análise de sentimento** nesses textos.


> Obs.: no Google Colab o `transformers` normalmente não vem instalado por padrão,
então eu coloquei um `pip install` aqui embaixo. Se já estiver instalado no ambiente,
essa célula pode ser ignorada ou adaptada.


In [ ]:
# instalação das bibliotecas necessárias (especialmente transformers)
# no Google Colab isso costuma funcionar direto.
# Se der algum aviso de "Restart runtime", é só rodar de novo a partir daqui.

!pip install -q transformers torch

In [ ]:
# Bloco de imports principais.

import os
import tarfile
import urllib.request
from pathlib import Path
import random

import numpy as np
import matplotlib.pyplot as plt

from transformers import pipeline

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

random.seed(42)
np.random.seed(42)

print("Versão do NumPy:", np.__version__)

## 2. Download e extração da base IMDB

Aqui eu faço o download do arquivo `aclImdb_v1.tar.gz` usando o link indicado no enunciado
e extraio o conteúdo para uma pasta local.

Confesso que na primeira vez que mexi com esse dataset eu apanhei um pouco com o `tarfile`,
então deixei esta parte mais comentada para lembrar o passo a passo.


In [ ]:
url_imdb = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
arquivo_tar = Path("aclImdb_v1.tar.gz")
pasta_destino = Path("aclImdb")

# Download do arquivo, se ainda não existir
if not arquivo_tar.exists():
    print(f"Baixando dataset IMDB de {url_imdb} ...")
    urllib.request.urlretrieve(url_imdb, arquivo_tar)
    print("Download concluído.")
else:
    print("Arquivo .tar.gz já existe, não vou baixar de novo.")

# Extração do tar.gz
if not pasta_destino.exists():
    print(f"Extraindo {arquivo_tar} ...")
    with tarfile.open(arquivo_tar, "r:gz") as tar:
        # Aqui eu confio no dataset, mas em geral é bom tomar cuidado com path traversal.
        tar.extractall()
    print("Extração concluída.")
else:
    print("Pasta aclImdb já existe, não vou extrair novamente.")

## 3. Carregando uma amostra de reviews (train/test, pos/neg)

A base completa é relativamente grande (25.000 reviews positivos + 25.000 negativos em treino,
e o mesmo em teste). Rodar o `pipeline` em tudo isso pode ser um pouco pesado, principalmente
se eu estiver sem GPU.

Então, para este trabalho, eu vou carregar **apenas uma amostra** de cada classe, por exemplo:

- `n_amostras_por_classe_treino = 500`
- `n_amostras_por_classe_teste = 200`

Isso já é suficiente para ter uma ideia do desempenho do modelo, sem deixar o notebook muito lento.


In [ ]:
def carregar_reviews(pasta_base, split, rotulo_str, limite=None):
    """
    Carrega os textos de uma pasta do IMDB.

    Exemplo de caminho:
    aclImdb/train/pos
    aclImdb/train/neg
    aclImdb/test/pos
    aclImdb/test/neg

    rotulo_str: "pos" ou "neg"
    limite: se não for None, limita o número de arquivos carregados.
    """
    pasta = Path(pasta_base) / split / rotulo_str
    arquivos = list(pasta.glob("*.txt"))
    random.shuffle(arquivos)  # misturo um pouco para pegar uma amostra variada

    textos = []
    rotulos = []
    count = 0

    for caminho in arquivos:
        with open(caminho, "r", encoding="utf-8") as f:
            texto = f.read().strip()

        textos.append(texto)
        # vou usar 1 para positivo e 0 para negativo
        rotulos.append(1 if rotulo_str == "pos" else 0)
        count += 1

        if limite is not None and count >= limite:
            break

    return textos, rotulos


# parâmetros de amostragem
n_amostras_por_classe_treino = 500
n_amostras_por_classe_teste = 200

# treino
X_train_pos, y_train_pos = carregar_reviews(pasta_destino, "train", "pos", limite=n_amostras_por_classe_treino)
X_train_neg, y_train_neg = carregar_reviews(pasta_destino, "train", "neg", limite=n_amostras_por_classe_treino)

X_train = X_train_pos + X_train_neg
y_train = y_train_pos + y_train_neg

# teste
X_test_pos, y_test_pos = carregar_reviews(pasta_destino, "test", "pos", limite=n_amostras_por_classe_teste)
X_test_neg, y_test_neg = carregar_reviews(pasta_destino, "test", "neg", limite=n_amostras_por_classe_teste)

X_test = X_test_pos + X_test_neg
y_test = y_test_pos + y_test_neg

print("Tamanho treino:", len(X_train))
print("Tamanho teste :", len(X_test))

In [ ]:
# Em seguida, misturo os exemplos de treino e teste (apenas para não ficar tudo agrupado).
# Como não é série temporal, não tem problema embaralhar aqui.

def embaralhar(X, y):
    indices = list(range(len(X)))
    random.shuffle(indices)
    X_emb = [X[i] for i in indices]
    y_emb = [y[i] for i in indices]
    return X_emb, y_emb

X_train, y_train = embaralhar(X_train, y_train)
X_test, y_test = embaralhar(X_test, y_test)

print("Exemplo de review (treino):")
print(X_train[0][:500], "...")  # mostro só um pedaço para não poluir demais
print("Rótulo esperado:", y_train[0])

## 4. Criando o `pipeline` de análise de sentimento

Aqui eu uso a classe `pipeline` da biblioteca `transformers`, exatamente como mostrado
no material da HuggingFace.

Vou usar a tarefa `"sentiment-analysis"` com um modelo já treinado para inglês.
Para não complicar demais, escolhi o modelo padrão da própria função:

```python
analisador = pipeline("sentiment-analysis")
```

Isso já baixa um modelo compatível (normalmente algo como `distilbert-base-uncased-finetuned-sst-2-english`).


In [ ]:
# Criação do pipeline de análise de sentimento.
# Se estiver em um ambiente com GPU, o HuggingFace pode usar automaticamente,
# mas aqui eu não forcei nada para não ficar dependente disso.

analisador = pipeline("sentiment-analysis")

# Teste rápido em uma frase simples
print(analisador("This movie was surprisingly good, I really enjoyed it!"))

## 5. Aplicando o `pipeline` nos reviews (amostra)

Rodar o modelo em todos os reviews pode ser um pouco lento, então optei por usar
uma **amostra menor** também na etapa de inferência, por exemplo:

- 400 reviews de teste (ou o tamanho completo de `X_test`, se eu já tiver limitado antes).

Para cada review, o `pipeline` retorna algo do tipo:

```python
[{'label': 'POSITIVE', 'score': 0.998...}]
```

Eu vou converter:

- `POSITIVE` → 1
- `NEGATIVE` → 0

e depois comparar com os rótulos verdadeiros (`y_test`) para calcular a acurácia.


In [ ]:
# para não ficar muito pesado, posso limitar a quantidade de exemplos de teste aqui também
max_avaliar = min(400, len(X_test))

X_test_amostra = X_test[:max_avaliar]
y_test_amostra = y_test[:max_avaliar]

print("Quantidade de reviews que vou avaliar:", len(X_test_amostra))

# aplicando o pipeline (isso pode demorar um pouco dependendo do ambiente)
# resultados = analisador(X_test_amostra)

# aplicando o pipeline (isso pode demorar um pouco dependendo do ambiente)
# como alguns reviews são bem grandes, eu tive que ativar o "truncation=True"
# para o tokenizer cortar o texto no tamanho máximo que o modelo aceita (512 tokens).
# resultados = analisador(X_test_amostra, truncation=True)
resultados = analisador(
    X_test_amostra,
    truncation=True,   # corta para o limite do modelo
    max_length=512     # só por segurança, mas em geral o modelo já usa isso
)

# convertendo labels para 0/1
def label_para_int(label_str):
    if label_str.upper().startswith("POS"):
        return 1
    else:
        return 0

y_pred_amostra = [label_para_int(r["label"]) for r in resultados]

acc = accuracy_score(y_test_amostra, y_pred_amostra)
print(f"Acurácia na amostra de teste: {acc:.4f}")

Quantidade de reviews que vou avaliar: 400


## 6. Matriz de confusão e relatório de classificação

Além da acurácia, é interessante ver:

- **matriz de confusão** (quantos positivos/negativos o modelo acerta/erra);
- um **relatório de classificação** com precisão, revocação e f1-score.

Isso não foi pedido explicitamente no enunciado, mas ajuda a entender melhor o comportamento
do modelo na base IMDB.


In [ ]:
cm = confusion_matrix(y_test_amostra, y_pred_amostra)
print("Matriz de confusão:")
print(cm)

print("\nRelatório de classificação:")
print(classification_report(y_test_amostra, y_pred_amostra, target_names=["neg", "pos"]))

## 7. Olhando alguns exemplos de previsões

Por fim, gosto de inspecionar manualmente alguns reviews, vendo:

- o texto (ou pelo menos o começo do texto);
- o rótulo verdadeiro;
- o rótulo previsto pelo modelo.

Isso ajuda a entender em que tipo de frase o modelo costuma errar.


In [ ]:
def mostrar_exemplos(indices):
    for i in indices:
        print("=" * 80)
        print(f"Review {i}")
        print("Texto (início):")
        print(X_test_amostra[i][:400].replace("\n", " "))
        print(f"Rótulo verdadeiro: {y_test_amostra[i]}  (0 = neg, 1 = pos)")
        print(f"Rótulo previsto  : {y_pred_amostra[i]}")
        print()

# pego alguns índices aleatórios só para ilustrar
indices_exemplo = random.sample(range(len(X_test_amostra)), k=min(5, len(X_test_amostra)))
mostrar_exemplos(indices_exemplo)

## 8. Conclusão

Neste Trabalho 04, eu:

1. Usei a base **IMDB** de reviews de filmes (`aclImdb_v1.tar.gz`), que contém textos
   em inglês rotulados como positivos ou negativos.

2. Fiz o **download** e a **extração** do arquivo manualmente, trabalhando diretamente
   com a estrutura de pastas `train/test` e `pos/neg`.

3. Carreguei uma **amostra** de reviews para não deixar o notebook muito pesado e
   misturei os exemplos de cada classe.

4. Criei um **`pipeline` de análise de sentimento** com a biblioteca `transformers`
   do HuggingFace, utilizando a tarefa `"sentiment-analysis"` em um modelo pré-treinado.

5. Apliquei o pipeline sobre os textos de teste (amostra) e converti os rótulos
   `"POSITIVE"` / `"NEGATIVE"` para inteiros (`1` / `0`) para poder comparar com os
   rótulos originais da base IMDB.

6. Calculei a **acurácia**, matriz de confusão e um relatório de classificação,
   observando o desempenho do modelo na base utilizada.

7. Inspecionei alguns exemplos individuais para ter uma noção mais qualitativa
   dos acertos e erros.

O foco deste trabalho foi justamente usar a classe `pipeline` da forma mostrada
no material da HuggingFace (*Behind the pipeline*), mas aplicando em uma base de
dados diferente da apresentada na aula, conforme o enunciado do professor.
